In [1]:
from statistics import mean
import pandas as pd
import struct

In [2]:
filename="putty.txt"

In [3]:
# Open the binary file for reading

file_handler = open(filename, "rb")
new_line_char=b'\r\n'
important_input_vals=[]

In [4]:
#----------------------AccelerometerSampleRate
aux_byte_list=[]
data_byte_1 = file_handler.read(2)
num, = struct.unpack('>h', data_byte_1)
important_input_vals.append(num/100)
#advance after new line char
data_byte_1 = file_handler.read(2)
#-----------------------GyroscopeSampleRate
aux_byte_list=[]
data_byte_1 = file_handler.read(2)
num, = struct.unpack('>h', data_byte_1)
important_input_vals.append(num/100)
#advance after new line char
data_byte_1 = file_handler.read(2)

In [5]:
#--------------------------PDM_Frequency
aux_byte_list=[]
data_byte_1 = file_handler.read(2)
num, = struct.unpack('>h', data_byte_1)
important_input_vals.append(num)
#advance after new line char
data_byte_1 = file_handler.read(2)

#------------------------------label_time_switch
aux_byte_list=[]
data_byte_1 = file_handler.read(2)
num, = struct.unpack('>H', data_byte_1)
important_input_vals.append(num)
#advance after new line char
data_byte_1 = file_handler.read(2)

In [6]:
#----------------------------------NrChannels
aux_byte_list=[]
data_byte_1 = file_handler.read(1)
num=(int.from_bytes(data_byte_1, "little"))
important_input_vals.append(num)
#advance after new line char
data_byte_1 = file_handler.read(2)

#----------------------------------NrLabels
aux_byte_list=[]
data_byte_1 = file_handler.read(1)
num=(int.from_bytes(data_byte_1, "little"))
important_input_vals.append(num)
#advance after new line char
data_byte_1 = file_handler.read(2)

In [14]:
for im in important_input_vals:
    print(type(im))

<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [8]:
data_list=[]
counter_micro=0
counter_acc=0
counter_gyro=0
while (1):
    aux_val_list=[]

    #----------------------label
    data_byte_1 = file_handler.read(1)
    if len(data_byte_1)==0:
        break
    aux_val_list.append(int.from_bytes(data_byte_1, "little"))

    #----------------------iteration
    data_byte_1 = file_handler.read(1)
    aux_val_list.append(int.from_bytes(data_byte_1, "little"))

    #----------------------timestamp
    data_byte_1 = file_handler.read(4)
    aux_val_list.append(int.from_bytes(data_byte_1, "little"))

    #----------------------flags
    data_byte_1 = file_handler.read(1)
    aux_byte=(int.from_bytes(data_byte_1, "little"))
    flags=[]
    flags.append(bool((aux_byte >> 0) & 1))
    flags.append(bool((aux_byte >> 1) & 1))
    flags.append(bool((aux_byte >> 2) & 1))
    flags.append(bool((aux_byte >> 3) & 1))

    aux_val_list.append(flags)

    #------------------------acc
    if flags[2] is True:
        #--------------------acc_x
        data_byte_1 = file_handler.read(2)
        num, = struct.unpack('>h', data_byte_1)
        aux_val_list.append(num/100)
        #--------------------acc_y
        data_byte_1 = file_handler.read(2)
        num, = struct.unpack('>h', data_byte_1)
        aux_val_list.append(num/100)
        #--------------------acc_z
        data_byte_1 = file_handler.read(2)
        num, = struct.unpack('>h', data_byte_1)
        aux_val_list.append(num/100)

        counter_acc=counter_acc+1
    else:
        aux_val_list.append(None)
        aux_val_list.append(None)
        aux_val_list.append(None)
    #------------------------gyro
    if flags[1] is True:
        #--------------------gyro_x
        data_byte_1 = file_handler.read(2)
        num, = struct.unpack('>h', data_byte_1)
        aux_val_list.append(num/100)
        #--------------------gyro_y
        data_byte_1 = file_handler.read(2)
        num, = struct.unpack('>h', data_byte_1)
        aux_val_list.append(num/100)
        #--------------------gyro_z
        data_byte_1 = file_handler.read(2)
        num, = struct.unpack('>h', data_byte_1)
        aux_val_list.append(num/100)

        counter_gyro=counter_gyro+1
    else:
        aux_val_list.append(None)
        aux_val_list.append(None)
        aux_val_list.append(None)

    #---------------------------micro
    if flags[0] is True:
        micro_i=0   
        aux_micro_list=[]
        while(1):

            data_byte_1 = file_handler.read(1)
            data_byte_2 = file_handler.read(1)

            #only leaves when new line char because nrsamples per buffer can change and be different from the one said by arduino rp2040
            if data_byte_1+data_byte_2==new_line_char or len(data_byte_1)==0 or len(data_byte_2)==0 :
                break

            byte_array=data_byte_2+data_byte_1
            num, = struct.unpack('>h', byte_array)
            aux_micro_list.append(num)

            counter_micro=counter_micro+1
            micro_i=micro_i+1

        aux_val_list.append(aux_micro_list)
        aux_val_list.append(round(mean(aux_micro_list),2))
    else:
        aux_val_list.append(None)
        aux_val_list.append(None)

    data_list.append(aux_val_list)
    
    if not flags[0]:
    #read new line bytes    
        data_byte_1 = file_handler.read(1)
        data_byte_2 = file_handler.read(1)
    
print(counter_acc,counter_gyro,counter_micro)

997 997 94688


In [32]:
for i in range(200):
    print(data_list[i][9],type(data_list[i][9]))

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
10.72 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
0.5 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
-6.76 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.5 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.25 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.12 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.06 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.06 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.06 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.06 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.06 <class 'float'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
-0.06 <class 'flo

In [10]:
len_list=[]
for row in data_list:
    if (row[10])!=None:
        len_list.append(len(row[10]))
len_mean=round(mean(len_list),2)
print(len_mean)

32


In [11]:
headers=["label","iteration","timestamp","flags","ax","ay","az","gx","gy","gz","micro","micro_mean"]
df = pd.DataFrame(data_list, columns=headers)

In [13]:
print(df.to_markdown())

|      |   label |   iteration |   timestamp | flags                       |   ax |     ay |     az |     gx |     gy |     gz | micro                                                                                                                                                                                                                            |   micro_mean |
|-----:|--------:|------------:|------------:|:----------------------------|-----:|-------:|-------:|-------:|-------:|-------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------:|
|    0 |       0 |           0 |        3073 | [True, False, False, False] |  nan | nan    | nan    | nan    | nan    | nan    | [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]                                 